In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.transforms as tfs
from scipy.signal import wiener
import collections
from scipy import stats
from matplotlib.ticker import MaxNLocator
import json  
import os
import seaborn as sns
import scikit_posthocs as sp
from IPython.display import clear_output
import networkx as nx
from bct.utils import BCTParamError, normalize, get_rng
import powerlaw
import warnings
import matplotlib
from scipy.spatial.distance import pdist,squareform
from scipy.cluster.hierarchy import linkage,dendrogram,fcluster
from statsmodels.sandbox.stats.runs import runstest_1samp 
import math
warnings.filterwarnings('ignore')

from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import umap.plot
from IPython.core.display import display, HTML
from pyrqa.time_series import TimeSeries
from pyrqa.settings import Settings
from pyrqa.analysis_type import Classic
from pyrqa.neighbourhood import FixedRadius, RadiusCorridor
from pyrqa.metric import EuclideanMetric, MaximumMetric, TaxicabMetric
from pyrqa.computation import RQAComputation
from pyrqa.computation import RPComputation
from pyrqa.image_generator import ImageGenerator
import umap
from pyrqa.neighbourhood import Unthresholded
plt.rcParams['svg.fonttype'] = 'none'

display(HTML("<style>.container { width:95% !important; }</style>"))

In [ ]:
def print_medias_sem(parameter):
    print('{:.3f}'.format(df[df['Condition']=='Control'][parameter].mean()),'±','{:.3f}'.format(2*df[df['Condition']=='Control'][parameter].sem()))
    print('{:.3f}'.format(df[df['Condition']=='Decorticated'][parameter].mean()),'±','{:.3f}'.format(2*df[df['Condition']=='Decorticated'][parameter].sem()))
    print('{:.3f}'.format(df[df['Condition']=='Parkinson'][parameter].mean()),'±','{:.3f}'.format(2*df[df['Condition']=='Parkinson'][parameter].sem()))
    print('{:.3f}'.format(df[df['Condition']=='Dyskinesia'][parameter].mean()),'±','{:.3f}'.format(2*df[df['Condition']=='Dyskinesia'][parameter].sem()))

In [ ]:
def plot_rate(raster,ci,c,bin_=4,fps=4,r=1.5):
    N,F=raster.shape
    indices_ci=np.where(ci==c)[0]
    raster_cluster=raster[indices_ci,:]
    y=np.sum(raster_cluster,axis=0)
    rate=np.zeros(y.shape)
    bin_b=0
    for j in range(y.shape[0]):
        rate[j]=np.sum(y[j-bin_b:j+1])
        if bin_b<bin_:
            bin_b+=1
    fpm=fps*60
    x=np.arange(0,F)/fpm
    fig=plt.figure(figsize=(12,3))
    #ax=plt.axes((0.05,0.35,0.75,0.9))
    plt.plot(x,rate,color='black')
    plt.xlim(np.min(x),np.max(x))
    plt.ylim(0,np.max(rate)+1)
    rate[np.where(rate==0)[0]]=np.nan
    time_series = TimeSeries(rate,embedding_dimension=2,time_delay=1)
    settings = Settings(time_series,
                    analysis_type=Classic,
                    neighbourhood=FixedRadius(r),    
                    #neighbourhood=Unthresholded(),
                    similarity_measure=EuclideanMetric,
                    theiler_corrector=1)
    computation = RPComputation.create(settings)
    result = computation.run()
    return result.recurrence_matrix

In [ ]:
def get_transition_graph(transition_matrix,cmap):
    G = nx.DiGraph()
    color_map=[]
    for i in range(len(transition_matrix)):
        G.add_node(i)
        if type(cmap)==matplotlib.colors.LinearSegmentedColormap:
            color_map.append(cmap(i/(len(transition_matrix)-1)))
        else:
            color_map.append(cmap(i))
    for i in range(len(transition_matrix)):
        for j in range(len(transition_matrix)):
            if transition_matrix[i,j]>=1:
                if type(cmap)==matplotlib.colors.LinearSegmentedColormap:
                    G.add_edge(i,j, weight=transition_matrix[i,j],color=cmap(i/(len(transition_matrix)-1))) 
                else:
                    G.add_edge(i,j, weight=transition_matrix[i,j],color=cmap(i))    
    return G,color_map

In [ ]:
def get_gephi_graph(ma,cluster_index,th):
    ma_reduced=threshold_proportional(ma, th)
    G=nx.from_numpy_matrix(ma_reduced)
    ensemble_attr_dict={}
    color_attr_dict={}
    cmap = matplotlib.cm.get_cmap('tab10')
    for i,ci in enumerate(cluster_index):
        ensemble_attr_dict[i]=str(ci)
        v=list(cmap(ci)[0:3])
        v=[vi*255 for vi in v]
        l=len(str(v))-1
        color_attr_dict[i]=str(v)[1:l]
    nx.set_node_attributes(G, ensemble_attr_dict, "ensemble")
    nx.set_node_attributes(G, color_attr_dict,"color")
    return G

In [ ]:
def get_ordered_adj_matrix(ma,cluster_index):
    ma_ordered=np.zeros(ma.shape)
    count=0
    new_order=[]
    for ci in range(max(cluster_index)+1):
        indices_ci=np.where(cluster_index==ci)[0]
        for indice in indices_ci:
            new_order.append(indice)
            count+=1
    for i in range(len(new_order)):
        indices_i=np.where(ma[new_order[i],:]>0)[0]
        for indice in indices_i:
            ma_ordered[i,np.where(new_order==indice)[0]]=ma[new_order[i],indice]
    
    return ma_ordered

In [ ]:
def get_adj_matrix(mapper):
    ma=np.zeros(mapper.graph_.shape)
    contador=0
    for i in range(ma.shape[0]):
        tam=mapper.graph_.indptr[i+1]-mapper.graph_.indptr[i]
        ma[i,mapper.graph_.indices[contador:contador+tam]]=mapper.graph_.data[contador:contador+tam]
        contador+=tam
    return ma

In [ ]:
def get_transition_matrix(colores,ci):
    secuencia=[-1]
    for key,values in colores.items():
        if len(values)>=1:
            for value in values:
                if secuencia[-1]!=value:
                    secuencia.append(value)
                    break
    del secuencia[0]
    transition_matrix=np.zeros((max(secuencia)+1,max(secuencia)+1))
    for i in range(len(secuencia)-1):
        transition_matrix[secuencia[i],secuencia[i+1]]+=1
    for i in range(max(ci)+1):
        #transition_matrix[i,:]=(transition_matrix[i,:]/np.sum(transition_matrix[i,:]))*100
        transition_matrix[i,:]=(transition_matrix[i,:]/np.sum(transition_matrix))*100
    return transition_matrix

In [ ]:
def get_color(colorRGBA1, colorRGBA2):
    red   = (colorRGBA1[0]  + colorRGBA2[0]) / 2
    green = (colorRGBA1[1]  + colorRGBA2[1]) / 2
    blue  = (colorRGBA1[2]  + colorRGBA2[2]) / 2
    return (red, green, blue, 1)

In [ ]:
def get_rates_ensembles(raster,ci,bin_=4,fps=4):
    N,F=raster.shape
    rates=np.zeros((raster.shape[1],max(ci)+1))
    for c in range(max(ci)+1):
        indices_ci=np.where(ci==c)[0]
        raster_cluster=raster[indices_ci,:]
        y=np.sum(raster_cluster,axis=0)
        rate=np.zeros(y.shape)
        bin_b=0
        for j in range(y.shape[0]):
            rate[j]=np.sum(y[j-bin_b:j+1])
            if bin_b<bin_:
                bin_b+=1
        rates[:,c]=rate
    fpm=fps*60
    x=np.arange(0,F)/fpm
    return rates

In [ ]:
def save_experiment_data(df,path_df):
    try:
        df_=pd.read_excel(path_df)
    except:
        df_=pd.DataFrame(columns=['Experiment','Condition','Num_Neurons','Num_Ensembles'])
    result = pd.concat([df_,df])
    result.to_excel(path_df, sheet_name='datos',index=False)

In [ ]:
def save_recurrence_data(df,path_df,exp_name,cond):
    df['Experiment']=exp_name
    df['Condition']=cond
    try:
        df_=pd.read_excel(path_df)
    except:
        df_=pd.DataFrame(columns=['Ensemble','N','RR','DET','L','L_max','DIV','L_entr','LAM','TT','V_max','V_entr','W','W_max','W_div','W_entr','Experiment','Condition'])
    result = pd.concat([df_,df])
    result.to_excel(path_df, sheet_name='datos',index=False)

In [ ]:
def plot_resume_recurrence(df):
    fig, axes = plt.subplots(1, 14,figsize=(14,4))
    sns.boxplot(data=df['RR'], orient='v', ax=axes[0]).set(title='RR')
    sns.boxplot(data=df['DET'], orient='v', ax=axes[1]).set(title='DET')
    sns.boxplot(data=df['L'], orient='v', ax=axes[2]).set(title='L')
    sns.boxplot(data=df['L_max'], orient='v', ax=axes[3]).set(title='L_max')
    sns.boxplot(data=df['DIV'], orient='v', ax=axes[4]).set(title='DIV')
    sns.boxplot(data=df['L_entr'], orient='v', ax=axes[5]).set(title='L_entr')
    sns.boxplot(data=df['LAM'], orient='v', ax=axes[6]).set(title='LAM')
    sns.boxplot(data=df['TT'], orient='v', ax=axes[7]).set(title='TT')
    sns.boxplot(data=df['V_max'], orient='v', ax=axes[8]).set(title='V_max')
    sns.boxplot(data=df['V_entr'], orient='v', ax=axes[9]).set(title='V_entr')
    sns.boxplot(data=df['W'], orient='v', ax=axes[10]).set(title='W')
    sns.boxplot(data=df['W_max'], orient='v', ax=axes[11]).set(title='W_max')
    sns.boxplot(data=df['W_div'], orient='v', ax=axes[12]).set(title='W_div')
    sns.boxplot(data=df['W_entr'], orient='v', ax=axes[13]).set(title='W_entr')
    fig.tight_layout()
    plt.show()

In [ ]:
def get_tam_pico_sig(raster,window_percentage=20,n_sigma=2):
    column = np.sum(raster,axis=0)
    N = len(column)
    time = np.arange(0,N)
    k = int(len(column) * (window_percentage/100))
    get_bands = lambda column : (np.mean(column) + n_sigma*np.std(column),np.mean(column) - n_sigma*np.std(column))
    bands = [get_bands(column[range(0 if i - k < 0 else i-k ,i + k if i + k < N else N)]) for i in range(0,N)]
    upper, lower = zip(*bands)
    anomalies = (column > upper) | (column < lower)
    
    for j in range(N):
        if anomalies[j]==True:
            if column[j]<2:
                anomalies[j]=False
    
    return anomalies,upper

In [ ]:
def test_coactivity_is_random(raster,ci,n_iter):
    for i in range(max(ci)+1):
        indices_ci=np.where(ci==i)[0]
        raster_cluster=raster[indices_ci,:]
        print('Cluster',i)
        coactivity_is_random(raster_cluster,n_iter)
        print()

In [ ]:
def plot_coactivity_ensambles(raster,cluster_index,n_iter,name_colormap,fps,window_percentage=20,n_sigma=2):
    Ci=cluster_index
    cmap = matplotlib.cm.get_cmap(name_colormap)
    N,F=raster.shape
    nmodules=max(Ci)+1
    maxCo=0

    for i in range(int(nmodules)):
        tempCo=np.sum(raster[np.where(Ci==i)[0],:],axis=0)
        if max(tempCo)>maxCo:
            maxCo=max(tempCo)

    fpm=fps*60
    x=np.arange(0,F)/fpm

    Co=np.zeros((int(nmodules),len(tempCo)))

    fig=plt.figure(figsize=(12,6))
    #ax=plt.axes((0.05,0.35,0.75,0.9))
    plt.subplots_adjust(hspace=0.000)
    number_of_subplots=nmodules
    ejes=[]
    for i in range(int(nmodules)):
        indices_ci=np.where(Ci==i)[0]
        raster_cluster=raster[indices_ci,:]
        peaks,th=get_tam_pico_sig(raster_cluster,window_percentage,n_sigma)     
        Co[i,:]=np.sum(raster[np.where(Ci==i)[0],:],axis=0)
        ax = plt.subplot(number_of_subplots,1,number_of_subplots-i)
        
        if type(cmap)==matplotlib.colors.LinearSegmentedColormap:
            ax.plot(x,Co[i,:],color=cmap(i/max(Ci)),alpha=1)
            ax.plot(x,th,color=cmap(i/max(Ci)),alpha=0.2)
            plt.fill_between(x, th, np.zeros(x.shape),facecolor=cmap(i/max(Ci)),alpha=0.1)
        else:        
            ax.plot(x,Co[i,:],color=cmap(i),alpha=1)
            ax.plot(x,th,color=cmap(i),alpha=0.2)
            plt.fill_between(x, th, np.zeros(x.shape),facecolor=cmap(i),alpha=0.1)

        ax.set_xlim(np.min(x),np.max(x))
        ax.set_ylim(0,maxCo+1)
        if i==0:
            plt.xlabel("Tiempo (min)")
            plt.ylabel("Coactividad")
        else:
            ax.set_xticks([])
        ejes.append(ax)

    for ax in ejes:
        pos=ax.get_position()
        pos=pos.from_extents(0.05,pos.y0,0.8,pos.y1)
        ax.set_position(pos)

In [ ]:
def plot_ensamble_raster(raster,fps,cluster_index,name_colormap,n_sigma=2,window_percentage=20,markersize=5):
    cmap = matplotlib.cm.get_cmap(name_colormap)
    N,F=raster.shape
    actividad=np.zeros((N,1))
    orden_plot=[]
    
    plt.figure(figsize=(12,6))

    ax=plt.axes((0.05,0.35,0.75,0.6))
    count=0
    ensembles_in_time={}
    for i in range(raster.shape[1]):
        ensembles_in_time[i]=[]
    
    for ci in range(max(cluster_index)+1):
        indices_ci=np.where(cluster_index==ci)[0]
        coactividad_cluster=np.sum(raster[np.where(cluster_index==ci)[0],:],axis=0)
        peaks,th=get_tam_pico_sig(raster[np.where(cluster_index==ci)[0],:],window_percentage,n_sigma)
        
        idx=np.where(peaks==True)[0]
        for i in indices_ci:
            orden_plot.append(i)
            indices=np.where(raster[i,:]==1)[0]
            
            if type(cmap)==matplotlib.colors.LinearSegmentedColormap:
                plt.plot(indices,raster[i,indices]*(count+1),
                        marker='|',linestyle='None',
                        markersize=markersize,color=cmap(ci/max(cluster_index)),alpha=0.1)
                plt.plot(idx,raster[i,idx]*(count+1),
                        marker='s',linestyle='None',
                        markersize=markersize,color=cmap(ci/max(cluster_index)),alpha=1)
            else:
                plt.plot(indices,raster[i,indices]*(count+1),
                        marker='|',linestyle='None',
                        markersize=markersize,color=cmap(ci),alpha=0.1)
                plt.plot(idx,raster[i,idx]*(count+1),
                        marker='s',linestyle='None',
                        markersize=markersize,color=cmap(ci),alpha=1)
            actividad[count]=np.sum(raster[i,:])*100/F
            count+=1
        
        for j,pk in enumerate(peaks):
            if pk:
                ensembles_in_time[j].append(ci)       
        
    ax.set_xlim(0,F-1)
    ax.set_ylim(1,N)
    plt.xticks([])
    plt.ylabel("Etiqueta de Neurona")

    ax=plt.axes(((0.05,0.12,0.75,0.2)))
    coactividad=np.sum(raster,axis=0)
    fpm=fps*60
    tiempo=np.arange(0,F)/fpm
    plt.plot(tiempo,coactividad,linewidth=0.5,color='black',alpha=1)
    ax.set_xlim(np.min(tiempo),np.max(tiempo))
    ax.set_ylim(0,ymax=np.max(coactividad)+1)
    plt.xlabel("Tiempo (min)")
    plt.ylabel("Coactividad")
    ymax=np.max(coactividad)+1
    inicio=-1
    final=-1
    bin_=markersize+2
    for ci in range(max(cluster_index)+1):
        for key,value in ensembles_in_time.items():
            encontre=False
            if len(value)>0:
                for ensemble in value:
                    if ensemble==ci:
                        encontre=True
                        if inicio<0:
                            inicio=key
                            final=key
                        else:
                            final=key
                        break
                if encontre==False:
                    if final>0:
                        if inicio-bin_<0:
                            if final+bin_>F:
                                x=tiempo[0:F]
                            else:
                                x=tiempo[0:final+bin_]
                        elif final+bin_>F:
                            x=tiempo[inicio-bin_:F]
                        else:
                            x=tiempo[inicio-bin_:final+bin_]
                        if type(cmap)==matplotlib.colors.LinearSegmentedColormap:
                            plt.fill_between(x, np.ones(x.shape)*ymax, np.zeros(x.shape),facecolor=cmap(ci/max(cluster_index)),alpha=0.5)
                        else:
                            plt.fill_between(x, np.ones(x.shape)*ymax, np.zeros(x.shape),facecolor=cmap(ci),alpha=0.5)
                        inicio=-1
                        final=-1
            if encontre==False:
                if final>0:
                    if inicio-bin_<0:
                        if final+bin_>F:
                            x=tiempo[0:F]
                        else:
                            x=tiempo[0:final+bin_]
                    elif final+bin_>F:
                        x=tiempo[inicio-bin_:F]
                    else:
                        x=tiempo[inicio-bin_:final+bin_]
                    if type(cmap)==matplotlib.colors.LinearSegmentedColormap:
                        plt.fill_between(x, np.ones(x.shape)*ymax, np.zeros(x.shape),facecolor=cmap(ci/max(cluster_index)),alpha=0.5)
                    else:
                        plt.fill_between(x, np.ones(x.shape)*ymax, np.zeros(x.shape),facecolor=cmap(ci),alpha=0.5)
                    inicio=-1
                    final=-1
                    
                    
    
    ax=plt.axes((0.85,0.35,0.1,0.6))
    plt.plot(actividad,np.arange(1,N+1),color='black',linewidth=1)
    ax.set_xlim(0,max(actividad)+1)
    ax.set_ylim(1,N)
    plt.xlabel("Cuadros activos")
    plt.yticks([])
    return orden_plot,ensembles_in_time

In [1]:
def get_2D_projection_vectors(raster,ci,n_cmap,window_percentage,n_sigma,min_dist,random_state,metric):
    cmap = matplotlib.cm.get_cmap(n_cmap)  
    colores={}
    for i in range(raster.shape[1]):
        colores[i]=[]
    for i in range(max(ci)+1):
        indices_ci=np.where(ci==i)[0]
        raster_cluster=raster[indices_ci,:]    
        peaks,th=get_tam_pico_sig(raster_cluster,window_percentage,n_sigma)
        for j,pk in enumerate(peaks):
            if pk:
                colores[j].append(i)
                
    raster_ensemble=np.empty((raster.shape[0],0), int)
    color_to_plot=[]
    count=0
    for i in range(raster.shape[1]):
        list_=colores[i]
        if len(list_)>0:
            for item in list_:
                raster_ensemble=np.append(raster_ensemble, np.expand_dims(raster[:,i], axis=1), axis=1)
                raster_ensemble[np.where(ci!=item)[0],count]=0
                color_to_plot.append(item)
                count+=1

    mapper = umap.UMAP(min_dist=min_dist,n_components=2,random_state=random_state,metric=metric).fit(raster_ensemble.T)

    fig = plt.figure()
    ax = fig.add_subplot(111)
    for i in range(raster_ensemble.shape[1]):
        ax.scatter(mapper.embedding_[i, 0], mapper.embedding_[i, 1], s=25, color=cmap(color_to_plot[i]),alpha=0.6)
    
#     mapper = umap.UMAP(min_dist=min_dist,n_components=2,random_state=random_state,metric=metric).fit(raster.T)
#     fig = plt.figure()
#     ax = fig.add_subplot(111)
#     sin_actividad=np.where(np.sum(raster,axis=0)<2)[0]
#     for i in range(raster.shape[1]):
#         if i not in sin_actividad:
#             if len(colores[i])>0:
#                 ax.scatter(mapper.embedding_[i, 0], mapper.embedding_[i, 1], s=25, color=cmap(colores[i][0]))
        

In [ ]:
def get_3D_projection_vectors(raster,ci,n_cmap,window_percentage,n_sigma,min_dist,random_state,metric):
    #%matplotlib notebook
    cmap = matplotlib.cm.get_cmap(n_cmap)  
    colores={}
    for i in range(raster.shape[1]):
        colores[i]=[]
    for i in range(max(ci)+1):
        indices_ci=np.where(ci==i)[0]
        raster_cluster=raster[indices_ci,:]    
        peaks,th=get_tam_pico_sig(raster_cluster,window_percentage,n_sigma)
        for j,pk in enumerate(peaks):
            if pk:
                colores[j].append(i)
    raster_ensemble=np.empty((raster.shape[0],0), int)
    color_to_plot=[]
    count=0
    for i in range(raster.shape[1]):
        list_=colores[i]
        if len(list_)>0:
            for item in list_:
                raster_ensemble=np.append(raster_ensemble, np.expand_dims(raster[:,i], axis=1), axis=1)
                raster_ensemble[np.where(ci!=item)[0],count]=0
                color_to_plot.append(item)
                count+=1            
    
    mapper = umap.UMAP(min_dist=min_dist,n_components=3,random_state=random_state,metric=metric).fit(raster_ensemble.T)
    
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    for i in range(raster_ensemble.shape[1]):
        if type(cmap)==matplotlib.colors.LinearSegmentedColormap:
            ax.scatter(mapper.embedding_[i, 0], mapper.embedding_[i, 1],mapper.embedding_[i, 2],
                       s=15, color=cmap(color_to_plot[i]/max(color_to_plot)))
        else:
            ax.scatter(mapper.embedding_[i, 0], mapper.embedding_[i, 1],mapper.embedding_[i, 2],
                       s=15, color=cmap(color_to_plot[i]))
    
    return mapper,color_to_plot,colores

In [ ]:
def get_recurrence_analysis(raster,ci,r=1.5,bin_=4):
    df=pd.DataFrame(columns=['Ensemble','N','RR','DET','L','L_max','DIV','L_entr','LAM','TT','V_max','V_entr','W','W_max','W_div','W_entr'])
    for i in range(max(ci)+1):
        indices_ci=np.where(ci==i)[0]
        raster_cluster=raster[indices_ci,:]
        y=np.sum(raster_cluster,axis=0)
        rate=np.zeros(y.shape)
        bin_b=0
        for j in range(y.shape[0]):
            rate[j]=np.sum(y[j-bin_b:j+1])
            if bin_b<bin_:
                bin_b+=1
        rate[np.where(rate==0)[0]]=np.nan
        time_series = TimeSeries(rate,embedding_dimension=2,time_delay=1)

        settings = Settings(time_series,
                            analysis_type=Classic,
                            neighbourhood=FixedRadius(r),
                            similarity_measure=EuclideanMetric,
                            theiler_corrector=1)
        computation = RQAComputation.create(settings,
                                        verbose=True)

        result = computation.run()
        result.min_diagonal_line_length = 2
        result.min_vertical_line_length = 2
        result.min_white_vertical_line_length = 2
        fila={'Ensemble':i,'N':len(indices_ci),
              'RR':result.recurrence_rate,'DET':result.determinism,
              'L':result.average_diagonal_line,'L_max':result.longest_diagonal_line,'DIV':result.divergence,
              'L_entr':result.entropy_diagonal_lines,'LAM':result.laminarity,
              'TT':result.laminarity,'V_max':result.longest_vertical_line,
              'V_entr':result.entropy_vertical_lines,'W':result.average_white_vertical_line,
              'W_max':result.longest_white_vertical_line,'W_div':result.longest_white_vertical_line_inverse,
              'W_entr':result.entropy_white_vertical_lines}
        df=df.append(fila, ignore_index=True)
        clear_output()

    # v_max=0
    # for i in range(max(ci)+1):
    #     indices_ci=np.where(ci==i)[0]
    #     raster_cluster=raster[indices_ci,:]
    #     y=np.sum(raster_cluster,axis=0)
    #     #y[np.where(y<tam_pico_sig[i])]=0
    #     rate=np.zeros(y.shape)
    #     bin_b=0
    #     for j in range(y.shape[0]):
    #         rate[j]=np.sum(y[j-bin_b:j+1])
    #         if bin_b<bin_:
    #             bin_b+=1
    #     time_series = TimeSeries(rate,embedding_dimension=2,time_delay=1)
    #     settings = Settings(time_series,
    #                     analysis_type=Classic,
    #                     neighbourhood=FixedRadius(1.5),
    #                     #neighbourhood=Unthresholded(),
    #                     similarity_measure=EuclideanMetric,
    #                     theiler_corrector=1)
    #     computation = RPComputation.create(settings)
    #     result = computation.run()
    #     if np.max(result.recurrence_matrix)>v_max:
    #         v_max=np.max(result.recurrence_matrix)
    # v_max=math.ceil(v_max)
    v_max=1
    for i in range(max(ci)+1):
        indices_ci=np.where(ci==i)[0]
        raster_cluster=raster[indices_ci,:]
        y=np.sum(raster_cluster,axis=0)
        rate=np.zeros(y.shape)
        bin_b=0
        for j in range(y.shape[0]):
            rate[j]=np.sum(y[j-bin_b:j+1])
            if bin_b<bin_:
                bin_b+=1
        rate[np.where(rate==0)[0]]=np.nan
        time_series = TimeSeries(rate,embedding_dimension=2,time_delay=1)
        settings = Settings(time_series,
                        analysis_type=Classic,
                        neighbourhood=FixedRadius(r),    
                        #neighbourhood=Unthresholded(),
                        similarity_measure=EuclideanMetric,
                        theiler_corrector=1)
        computation = RPComputation.create(settings)
        result = computation.run()
        plt.matshow(result.recurrence_matrix,cmap='Blues',vmin=0, vmax=v_max)
        plt.colorbar()
    
    return df

In [ ]:
def coactivity_is_random(raster,n_iter):
    coactividad=np.sum(raster,axis=0)
    rt,p=runstest_1samp(coactividad,cutoff='mean',correction=False)
    print('rt =',rt)
    print('p =',p)
    #type 1 error
    Im=0
    for i in range(n_iter):
        raster_s=raster_subrogado(raster,1)
        coactividad_s=np.sum(raster_s,axis=0)
        rt_s,p_s=runstest_1samp(coactividad_s,cutoff='median',correction=False)
        if p_s<=0.05:
            Im+=1
    alphahat=Im/n_iter
    print('alphahat',alphahat)
    #type 2 error
    Im=0
    for i in range(n_iter):
        raster_s=raster_subrogado(raster,2)
        coactividad_s=np.sum(raster_s,axis=0)
        rt_s,p_s=runstest_1samp(coactividad_s,cutoff='median',correction=False)
        if p_s>0.05:
            Im+=1
    betahat=Im/n_iter
    print('betahat',betahat)
    return rt,p,alphahat,betahat

In [ ]:
def plot_histogram_isi(tiempo_spikes,n_bins=50):
    isi=np.diff(tiempo_spikes)
    plt.hist(isi, bins=n_bins, density=True, color='lightblue', edgecolor='black')
    plt.title('Histograma ISI')
    plt.xlabel('Intervalo Interespiga [s]')
    plt.ylabel('Densidad de Probabilidad')
    plt.tight_layout()

In [ ]:
def test_coactividad_significativa_clusters(raster_real,ci,n_iter,p): 
    tam_pico_sig=[]
    for i in range(max(ci)+1):
        indices_ci=np.where(ci==i)[0]
        raster_cluster=raster[indices_ci,:]
        hist_real_cum,lista_hist_cum_subrogada=coctivity_test(raster_cluster, n_iter)
        for j in range(len(lista_hist_cum_subrogada)):
            for k in range(len(hist_real_cum)-len(lista_hist_cum_subrogada[j])):
                lista_hist_cum_subrogada[j].append(0)
        for n in range(2,len(hist_real_cum)):
            vc=np.percentile([lista[n] for lista in lista_hist_cum_subrogada],100-p*100)
            if hist_real_cum[n]>vc:
                tam_pico_sig.append(n)
                break
    return tam_pico_sig

In [ ]:
def coctivity_test(raster_real, n_iter, opt):
    coactividad_real=np.sum(raster_real,axis=0)
    frecuencia_coactividad=collections.Counter(coactividad_real)
    hist_real_cum=[]
    for i in range(int(np.max(list(frecuencia_coactividad.keys()))+1)):
        suma=0
        for j in range(i,int(np.max(list(frecuencia_coactividad.keys()))+1)):
            suma+=frecuencia_coactividad[j]
        hist_real_cum.append(suma)

    lista_hist_subrogada=[]
    lista_hist_cum_subrogada=[]
    for i in range(n_iter):
        raster_sub=raster_subrogado(raster_real,opt)
        coactividad_subrogada=np.sum(raster_sub,axis=0)
        frecuencia_coactividad_subrogada=collections.Counter(coactividad_subrogada.astype(int))
        lista_hist_subrogada.append(frecuencia_coactividad_subrogada)
        hist_sub_cum=[]
        for i in range(int(np.max(list(frecuencia_coactividad_subrogada.keys()))+1)):
            suma=0
            for j in range(i,int(np.max(list(frecuencia_coactividad_subrogada.keys()))+1)):
                suma+=frecuencia_coactividad_subrogada[j]
            hist_sub_cum.append(suma)
        lista_hist_cum_subrogada.append(hist_sub_cum)
    return hist_real_cum,lista_hist_cum_subrogada

In [ ]:
def plot_coactividad_clusters_h(raster,Ci,name_colormap,fps,tam_pico_sig=None):
    cmap = matplotlib.cm.get_cmap(name_colormap)
    N,F=raster.shape
    nmodules=max(Ci)+1
    maxCo=0

    for i in range(nmodules):
        tempCo=np.sum(raster[np.where(Ci==i)[0],:],axis=0)
        if max(tempCo)>maxCo:
            maxCo=max(tempCo)

    fpm=fps*60
    x=np.arange(0,F)/fpm

    Co=np.zeros((nmodules,len(tempCo)))

    fig=plt.figure(figsize=(12,6))
    #ax=plt.axes((0.05,0.35,0.75,0.9))
    plt.subplots_adjust(hspace=0.000)
    number_of_subplots=nmodules
    ejes=[]
    for i in range(nmodules):
        Co[i,:]=np.sum(raster[np.where(Ci==i)[0],:],axis=0)
        ax = plt.subplot(number_of_subplots,1,number_of_subplots-i)
        ax.plot(x,Co[i,:],color=cmap(i))
        if len(tam_pico_sig)>0:
            ax.plot([min(x),max(x)],[tam_pico_sig[i]-0.2,tam_pico_sig[i]-0.2],linestyle='--',color='black',linewidth=1)
        ax.set_xlim(np.min(x),np.max(x))
        ax.set_ylim(0,maxCo+1)
        if i==0:
            plt.xlabel("Tiempo (min)")
            plt.ylabel("Coactividad")
        else:
            ax.set_xticks([])
        ejes.append(ax)

    for ax in ejes:
        pos=ax.get_position()
        pos=pos.from_extents(0.05,pos.y0,0.8,pos.y1)
        ax.set_position(pos)

In [ ]:
def coactividad_significativa_clusters_h(raster_real,ci,n_iter,p,graficar=False): 
    tam_pico_sig=[]
    for i in range(max(ci)+1):
        indices_ci=np.where(ci==i)[0]
        raster_cluster=raster[indices_ci,:]
        tam_picos,ps,_=test_coactividad_significativa(raster_cluster,n_iter,p,graficar)
        prueba=ps[2:]
        indice=np.nan
        for idx,valor in enumerate(prueba):
            if valor<p:
                indice=idx
                break
        tam_pico_sig.append(indice+2)
    return tam_pico_sig

In [ ]:
def test_error_tipo_1(raster_real,n_iter,n_ensayos,n_neu_pico,alpha):
    sumas_hipotesis=[]
    for ni in range(10000):
        raster_hipotesis=raster_subrogado(raster_real,1)
        coactividad_hipotesis=np.sum(raster_hipotesis,axis=0)
        frecuencia_coactividad=collections.Counter(coactividad_hipotesis)
        suma_hipotesis=0
        for i in range(n_neu_pico,len(frecuencia_coactividad)):
            suma_hipotesis+=frecuencia_coactividad[i]
        sumas_hipotesis.append(suma_hipotesis)
    valor_critico=np.quantile(sumas_hipotesis, 1-alpha)

    p_ensayos=[]
    for ne in range(n_ensayos):
        frecuencia_subrogada=[]
        for ni in range(n_iter):
            raster_sub=raster_subrogado(raster_hipotesis,1)
            coactividad_subrogada=np.sum(raster_sub,axis=0)
            frecuencia_coactividad_subrogada=collections.Counter(coactividad_subrogada.astype(int))
            suma_subr=0
            for i in range(n_neu_pico,len(frecuencia_coactividad_subrogada)):
                suma_subr+=frecuencia_coactividad_subrogada[i]
            frecuencia_subrogada.append(suma_subr)
        p_ensayos.append(len(np.where(valor_critico<np.array(frecuencia_subrogada))[0])/n_iter)
    plt.boxplot(p_ensayos)
    plt.title('Alpha error tipo 1')
    plt.show()
    
    return p_ensayos

In [ ]:
def test_error_tipo_2(raster_real,n_iter,n_ensayos,n_neu_pico,alpha):
    sumas_hipotesis=[]
    for ni in range(10000):
        raster_hipotesis=raster_subrogado(raster_real,2)
        coactividad_hipotesis=np.sum(raster_hipotesis,axis=0)
        frecuencia_coactividad=collections.Counter(coactividad_hipotesis)
        suma_hipotesis=0
        for i in range(n_neu_pico,len(frecuencia_coactividad)):
            suma_hipotesis+=frecuencia_coactividad[i]
        sumas_hipotesis.append(suma_hipotesis)
    valor_critico=np.quantile(sumas_hipotesis, 1-alpha)

    p_ensayos=[]
    for ne in range(n_ensayos):
        frecuencia_subrogada=[]
        for ni in range(n_iter):
            raster_sub=raster_subrogado(raster_hipotesis,1)
            coactividad_subrogada=np.sum(raster_sub,axis=0)
            frecuencia_coactividad_subrogada=collections.Counter(coactividad_subrogada.astype(int))
            suma_subr=0
            for i in range(n_neu_pico,len(frecuencia_coactividad_subrogada)):
                suma_subr+=frecuencia_coactividad_subrogada[i]
            frecuencia_subrogada.append(suma_subr)
        p_ensayos.append(len(np.where(valor_critico<np.array(frecuencia_subrogada))[0])/n_iter)
    plt.boxplot(p_ensayos)
    plt.title('Beta error tipo 2')
    plt.show()
    return p_ensayos

In [1]:
def get_clusters_modularidad(grafo_ponderado,n_iter):
    N=len(grafo_ponderado)
    mat_mismo_grupo=np.zeros(grafo_ponderado.shape)
    for i in range(n_iter):
        cluster_index,_=community_louvain(grafo_ponderado,seed=42)
        for neu in range(N):
            module = cluster_index[neu]
            index = np.where(cluster_index==module)[0]
            mat_mismo_grupo[neu,index]+=1
    for i in range(N):
        mat_mismo_grupo[i,i]=0
    cluster_index_h,_=community_louvain(mat_mismo_grupo,seed=42)    
    cluster_index_h=cluster_index_h-1
    print("Number of Ensembles: ",str(max(cluster_index_h)+1))
    return cluster_index_h,mat_mismo_grupo

In [ ]:
def test_mundo_pequeño(G,n_iter):
    n=len(G.nodes)
    k=len(G.edges)
    buscar=True
    knn=2
    while buscar:
      GA=nx.watts_strogatz_graph(n,knn,p=0)
      if len(GA.edges)>k:
        buscar=False
      else:
        knn+=2
    count=0
    L_L=[]
    L_R=[]
    C_L=[]
    C_R=[]
    while count < n_iter:
      GL = GA.copy()
      while len(GL.edges)>k:
        degrees_iter = np.array([GL.degree(n) for n in GL.nodes()])
        max_degree=max(degrees_iter)
        n_del=np.random.choice(np.where(degrees_iter==max_degree)[0])
        lista_enlaces=list(GL.edges)
        lista_quitar=[]
        for i in range(len(lista_enlaces)):
          if lista_enlaces[i][0]==n_del or lista_enlaces[i][1]==n_del:
            lista_quitar.append(i)
        elem_del=[]
        for i in range(len(lista_quitar)):
          if degrees_iter[lista_enlaces[lista_quitar[i]][0]]<max_degree or degrees_iter[lista_enlaces[lista_quitar[i]][1]]<max_degree:
            elem_del.append(i)
        delete_multiple_element(lista_quitar, elem_del)
        if len(lista_quitar)==0:
          continue
        edge_del=np.random.choice(lista_quitar)
        GL.remove_edge(lista_enlaces[edge_del][0],lista_enlaces[edge_del][1])
      GR = GL.copy()
      lista_enlaces=list(GR.edges)
      lista_nodos=list(GR.nodes)
      for enlace in lista_enlaces:
        lista_w=lista_nodos.copy()
        u=enlace[0]
        v=enlace[1]
        not_w=[u]
        for node in GR.edges(u):
          not_w.append(node[1])
        delete_multiple_element(lista_w, not_w)
        w=np.random.choice(lista_w)
        GR.remove_edge(u,v)
        GR.add_edge(u,w)
      if not nx.is_connected(GR):
        continue
      count+=1
      C_R.append(nx.average_clustering(GR))
      L_R.append(nx.average_shortest_path_length(GR))
      C_L.append(nx.average_clustering(GL))
      L_L.append(nx.average_shortest_path_length(GL))
    
    C=nx.average_clustering(G)
    L=nx.average_shortest_path_length(G)
    print("C = ",C)
    print("L = ",L)
    
    C_R_mean = np.mean(C_R)
    L_R_mean = np.mean(L_R)
    C_L_mean = np.mean(C_L)
    L_L_mean = np.mean(L_L)
    
    smallworldness_all=[]
    for i in range(n_iter):
      smallworldness_all.append((L/L_R[i])-(C/C_L[i]))
    smallworldness_mean=np.mean(smallworldness_all)
    print("smallworldness = ",smallworldness_mean)
    
    plt.figure()
    plt.hist(smallworldness_all,density=True)
    plt.title('Smallworldness')
    
    plt.figure()
    plt.boxplot([C_L,C,C_R])
    plt.ylabel("Coeficiente de Agrupamiento")
    plt.xticks([1, 2, 3], ['Regular', 'Real', 'Aleatoria'])
    
    plt.figure()
    plt.boxplot([L_L,L,L_R])
    plt.ylabel("Shortest path length")
    plt.xticks([1, 2, 3], ['Regular', 'Real', 'Aleatoria'])
    
    salida={'L':L,'C':C,'C_Laticce':C_L_mean,'L_Laticce':L_L_mean,'C_Random':C_R_mean,'L_Random':L_R_mean,
           'Smallworldness':smallworldness_mean}
    
    return salida

In [ ]:
def plot_distribucion_grados(G):
    degrees = np.array([G.degree(n) for n in G.nodes()])
    hist, bin_edge=np.histogram(degrees, bins=range(min(degrees),max(degrees)+2),density=True)
    plt.bar(bin_edge[:-1],hist,width=1)
    plt.xlim([0,max(degrees)+1])
    plt.gca().set_xticks(range(1,max(degrees)+1))
    plt.xlabel("Grado [k]")
    plt.ylabel("P(k)")
    plt.title("Distribución de Grados")
    plt.show()

In [ ]:
def test_similaridad_clusters(raster,cluster_index_h,cluster_index_v,umbral):
    N,F=raster.shape
    coactividad=np.sum(raster,axis=0)
    indices_coactividad_sig=np.where(coactividad>=umbral)[0]
    vectores_columna=raster[:,indices_coactividad_sig]
    informacion={}
    for ci in range(max(cluster_index_v)+1):
        informacion["cluster_"+str(ci)]={}
        ind_ci=np.where(cluster_index_v==ci)[0]
        for i in ind_ci:
            neu_act=np.where(vectores_columna[:,i]==1)[0]
            for neu in neu_act:
                if neu not in informacion["cluster_"+str(ci)]:
                    informacion["cluster_"+str(ci)][neu] = 1
                else:
                    informacion["cluster_"+str(ci)][neu] += 1    
    cluster_pref_neu=np.zeros((N,))
    cluster_pref_neu[:] = np.nan
    tam_pref_neu=np.zeros((N,))
    for n in range(N):
        count=0
        for key in informacion:
            if n in informacion[key]:
                if informacion[key][n]>tam_pref_neu[n]:
                    cluster_pref_neu[n]=count
                    tam_pref_neu[n]=informacion[key][n]
            count+=1
    orden_aparicion=[]
    for i in range(len(cluster_index_v)):
        if i==0:
            orden_aparicion.append(cluster_index_v[i])
        if cluster_index_v[i] not in orden_aparicion:
            orden_aparicion.append(cluster_index_v[i])
    orden_indices=[]
    for i in orden_aparicion:
        neu_ind_reorder=np.where(cluster_pref_neu==i)[0]
        orden_indices.append([neu_ind_reorder])
    neu_ind_reorder=np.argwhere(np.isnan(cluster_pref_neu))
    orden_indices.append([np.array([i[0] for i in neu_ind_reorder])])
    
    neu_clus_h={}
    for ci in range(max(cluster_index_h)+1):
        neu_clus_h["cluster_"+str(ci)]=[]
    for n in range(len(cluster_index_h)):
        neu_clus_h['cluster_'+str(cluster_index_h[n])].append(n)
        
    neu_clus_v={}
    for ci in range(int(max(cluster_pref_neu)+1)):
        neu_clus_v["cluster_"+str(ci)]=[]    
    for key,valor in informacion.items():
        for neu in valor:
            neu_clus_v[key].append(neu)
            
    mat_sobrelape_1=np.zeros((len(neu_clus_h),len(neu_clus_h)))    
    mat_sobrelape_2=np.zeros((len(neu_clus_h),len(neu_clus_h)))
    i=0
    for key_1,valor_1 in neu_clus_h.items():
        j=0
        for key_2,valor_2 in neu_clus_v.items():
            mat_sobrelape_1[i,j]=len(np.intersect1d(valor_1,valor_2))/len(valor_1)
            mat_sobrelape_2[j,i]=len(np.intersect1d(valor_1,valor_2))/len(valor_2)
            j+=1
        i+=1
    
    max_sim_1=[]
    for i in range(len(mat_sobrelape_1)):
        max_sim_1.append(max(mat_sobrelape_1[i,:])*100)
    max_sim_2=[]
    for i in range(len(mat_sobrelape_2)):
        max_sim_2.append(max(mat_sobrelape_2[i,:])*100)
    
    fig, (ax1, ax2) = plt.subplots(1, 2 , figsize=(12,8))
    
    im1 = ax1.matshow(mat_sobrelape_1,cmap='jet',vmin = 0, vmax = 1)
    fig.colorbar(im1, ax=ax1)
    ax1.set_title("Horizontal")
    ax1.xaxis.set_ticks_position('bottom')
    ax1.set_ylabel('Cluster Horizontal')
    ax1.set_xlabel('Cluster Vertical')
    
    im2 = ax2.matshow(mat_sobrelape_2,cmap='jet',vmin = 0, vmax = 1)
    fig.colorbar(im2, ax=ax2)
    ax2.set_title("Vertical")
    ax2.xaxis.set_ticks_position('bottom')
    ax2.set_ylabel('Cluster Vertical')
    ax2.set_xlabel('Cluster Horizontal')
    
    plt.figure(figsize=(10,5))  
    plt.boxplot([max_sim_1,max_sim_2])
    plt.xticks([1, 2], ['Horizontal', 'Vertical'])
    plt.ylabel("% inclusión")
    plt.ylim(-10,110)

In [ ]:
def plot_color_clusters(cluster_index,name_colormap):
    gradient = np.linspace(0, 10, 10)
    gradient = np.vstack((gradient, gradient))
    plt.figure(figsize=(5,1))
    plt.imshow(gradient, aspect='auto', cmap=plt.get_cmap(name_colormap))
    plt.xlim(-0.5,max(cluster_index)+0.5)
    plt.xlabel("Número del Cluster")
    plt.yticks([])
    plt.show()

In [ ]:
def find_nearest(array,value):
    idx = np.searchsorted(array, value, side="left")
    if idx > 0 and (idx == len(array) or math.fabs(value - array[idx-1]) < math.fabs(value - array[idx])):
        return array[idx-1]
    else:
        return array[idx]

In [ ]:
def plot_cluster_raster_v(raster,fps,cluster_index,name_colormap,umbral,smoothing=False,markersize=5):
    N,F=raster.shape
    coactividad=np.sum(raster,axis=0)
    indices_coactividad_sig=np.where(coactividad>=umbral)[0]
    vectores_columna=raster[:,indices_coactividad_sig]
    informacion={}
    for ci in range(max(cluster_index)+1):
        informacion["cluster_"+str(ci)]={}
        ind_ci=np.where(cluster_index==ci)[0]
        for i in ind_ci:
            neu_act=np.where(vectores_columna[:,i]==1)[0]
            for neu in neu_act:
                if neu not in informacion["cluster_"+str(ci)]:
                    informacion["cluster_"+str(ci)][neu] = 1
                else:
                    informacion["cluster_"+str(ci)][neu] += 1    
    cluster_pref_neu=np.zeros((N,))
    cluster_pref_neu[:] = np.nan
    tam_pref_neu=np.zeros((N,))
    for n in range(N):
        count=0
        for key in informacion:
            if n in informacion[key]:
                if informacion[key][n]>tam_pref_neu[n]:
                    cluster_pref_neu[n]=count
                    tam_pref_neu[n]=informacion[key][n]
            count+=1
    orden_aparicion=[]
    for i in range(len(cluster_index)):
        if i==0:
            orden_aparicion.append(cluster_index[i])
        if cluster_index[i] not in orden_aparicion:
            orden_aparicion.append(cluster_index[i])
    orden_indices=[]
    for i in orden_aparicion:
        neu_ind_reorder=np.where(cluster_pref_neu==i)[0]
        orden_indices.append([neu_ind_reorder])
    neu_ind_reorder=np.argwhere(np.isnan(cluster_pref_neu))
    orden_indices.append([np.array([i[0] for i in neu_ind_reorder])])
    raster_reorder=np.zeros(raster.shape)
    longitud=0
    while len(orden_indices)>(max(cluster_index)+1):
        orden_indices = orden_indices[:-1]
    for i in orden_indices:
        raster_reorder[range(longitud,longitud+len(i[0])),:]=raster[i[0],:]
        longitud+=len(i[0])
    
    cmap = matplotlib.cm.get_cmap(name_colormap)    
    plt.figure(figsize=(12,6))

    ax=plt.axes((0.05,0.35,0.75,0.6)) 
    for i in range(N):
        indices=np.where(raster_reorder[i,:]==1)[0]
        plt.plot(indices,raster_reorder[i,indices]*(i+1),
                marker='|',linestyle='None',
                markersize=markersize,color='black',alpha=0.1)

    indices_coactividad_sig=np.where(np.sum(raster_reorder,axis=0)>=umbral)[0]
    for ci in range(max(cluster_index)+1):
        raster_vec_col_cluster=np.zeros(raster_reorder.shape)
        vec_col_cluster=np.where(cluster_index==ci)[0]
        raster_vec_col_cluster[:,indices_coactividad_sig[vec_col_cluster]]=raster_reorder[:,indices_coactividad_sig[vec_col_cluster]]
        for i in range(N):
            indices=np.where(raster_vec_col_cluster[i,:]==1)[0]
            plt.plot(indices,raster_vec_col_cluster[i,indices]*(i+1),
                    marker='s',linestyle='None',
                    markersize=markersize,color=cmap(ci))

    ax.set_xlim(0,F-1)
    ax.set_ylim(1,N)
    plt.xticks([])
    plt.ylabel("Etiqueta de Neurona")

    ax=plt.axes(((0.05,0.12,0.75,0.2)))
    coactividad=np.sum(raster_reorder,axis=0)
    coactividad_smooth=smooth(coactividad,3)
    fpm=fps*60
    tiempo=np.arange(0,F)/fpm
    if umbral!=None:
        plt.plot([np.min(tiempo),np.max(tiempo)],[umbral,umbral],linestyle='--',color='black',linewidth=1)
    xvals = np.linspace(min(tiempo), max(tiempo), len(tiempo)*10)
    yinterp = np.interp(xvals, tiempo, coactividad)
    y_masked = np.ma.masked_where(yinterp < (umbral-0.5), yinterp)
    indices_coactividad_sig=np.where(coactividad>=umbral)[0]
    if smoothing==False:
        plt.plot(tiempo,coactividad,linewidth=0.5,color='black')
        for ci in range(max(cluster_index)+1):
            vec_col_cluster=np.where(cluster_index==ci)[0]
            y_masked_ci=y_masked.copy()
            y_masked_ci.mask=True
            for val in vec_col_cluster:
                tiempo_mas_cercano=tiempo[indices_coactividad_sig[val]]
                indice_pico=np.where(xvals==find_nearest(xvals, tiempo_mas_cercano))[0]
                y_masked_ci.mask[range(int(indice_pico)-5,int(indice_pico)+5)]=False
            plt.plot(xvals,y_masked_ci,linewidth=4,color=cmap(ci))
            #plt.plot(tiempo[indices_coactividad_sig[vec_col_cluster]],
            #         coactividad[indices_coactividad_sig[vec_col_cluster]],
            #         linestyle='None',marker='o',color=cmap(ci))
    else:
        plt.plot(tiempo,coactividad_smooth,linewidth=0.5,color='black')
    ax.set_xlim(np.min(tiempo),np.max(tiempo))
    ax.set_ylim(0,np.max(coactividad)+1)
    plt.xlabel("Tiempo (min)")
    plt.ylabel("Coactividad")

    ax=plt.axes((0.85,0.35,0.1,0.6))
    actividad=np.sum(raster_reorder,axis=1)
    plt.plot(actividad,np.arange(1,N+1),color='black',linewidth=1)
    ax.set_xlim(0,max(actividad)+1)
    ax.set_ylim(1,N)
    plt.xlabel("Cuadros activos")
    plt.yticks([])
    
    orden_plot=[]
    for ar in orden_indices:
        for vec in ar:
            for val in vec:
                orden_plot.append(val) 
    
    return orden_plot

In [ ]:
def smooth(y, box_pts):
    box = np.ones(box_pts)/box_pts
    y_smooth = np.convolve(y, box, mode='same')
    return y_smooth

In [ ]:
def nuevo_cluster_index(cluster_index,nuevo_orden):
    try:
        n=len(nuevo_orden)
        newCluster_index=np.zeros((len(cluster_index),))
        for i in range(max(cluster_index)+1):
            indices=np.where(cluster_index==nuevo_orden[i])[0]
            newCluster_index[indices]=int(i)
        return newCluster_index.astype(int)
    except ValueError:
        print(ValueError)
        return None

In [ ]:
def plot_cluster_raster_h(raster,fps,cluster_index,name_colormap,smoothing=False,markersize=5,tam_pico_sig=[]):
    cmap = matplotlib.cm.get_cmap(name_colormap)
    N,F=raster.shape
    actividad=np.zeros((N,1))
    orden_plot=[]
    if len(tam_pico_sig)==0:
        plt.figure(figsize=(12,6))

        ax=plt.axes((0.05,0.35,0.75,0.6))
        count=0
        for ci in range(max(cluster_index)+1):
            indices_ci=np.where(cluster_index==ci)[0]
            for i in indices_ci:
                orden_plot.append(i)
                indices=np.where(raster[i,:]==1)[0]
                if type(cmap)==matplotlib.colors.LinearSegmentedColormap:
                    plt.plot(indices,raster[i,indices]*(count+1),
                            marker='|',linestyle='None',
                            markersize=markersize,color=cmap(ci/max(cluster_index)))
                else:
                    plt.plot(indices,raster[i,indices]*(count+1),
                            marker='|',linestyle='None',
                            markersize=markersize,color=cmap(ci))
                actividad[count]=np.sum(raster[i,:])*100/F
                count+=1
        ax.set_xlim(0,F-1)
        ax.set_ylim(1,N)
        plt.xticks([])
        plt.ylabel("Etiqueta de Neurona")

        ax=plt.axes(((0.05,0.12,0.75,0.2)))
        coactividad=np.sum(raster,axis=0)
        fpm=fps*60
        tiempo=np.arange(0,F)/fpm

        coactividad_clusters=np.zeros((max(cluster_index),len(tiempo)))
        for ci in range(max(cluster_index)):
            coactividad_clusters[ci,:]=np.sum(raster[np.where(cluster_index==ci)[0],:],axis=0)

        if smoothing==False:
            for ci in range(max(cluster_index)):
                plt.plot(tiempo,coactividad_clusters[ci,:],linewidth=0.5,color=cmap(ci))
            plt.plot(tiempo,coactividad,linewidth=0.5,color='black')
        else:
            for ci in range(max(cluster_index)):
                plt.plot(tiempo,smooth(coactividad_clusters[ci,:],3),linewidth=0.5,color=cmap(ci))
            coactividad_smooth=smooth(coactividad,3)
            plt.plot(tiempo,coactividad_smooth,linewidth=0.5,color='black')
        ax.set_xlim(np.min(tiempo),np.max(tiempo))
        ax.set_ylim(0,np.max(coactividad)+1)
        plt.xlabel("Tiempo (min)")
        plt.ylabel("Coactividad")

        ax=plt.axes((0.85,0.35,0.1,0.6))
        plt.plot(actividad,np.arange(1,N+1),color='black',linewidth=1)
        ax.set_xlim(0,max(actividad)+1)
        ax.set_ylim(1,N)
        plt.xlabel("Cuadros activos")
        plt.yticks([])
    
    elif len(tam_pico_sig)==max(cluster_index)+1:
        print("dentro")
        plt.figure(figsize=(12,6))

        ax=plt.axes((0.05,0.35,0.75,0.6))
        count=0
        for ci in range(max(cluster_index)+1):
            indices_ci=np.where(cluster_index==ci)[0]
            coactividad_cluster=np.sum(raster[np.where(cluster_index==ci)[0],:],axis=0)
            idx=np.where(coactividad_cluster>=tam_pico_sig[ci])[0]
            for i in indices_ci:
                orden_plot.append(i)
                indices=np.where(raster[i,:]==1)[0]
                plt.plot(indices,raster[i,indices]*(count+1),
                        marker='|',linestyle='None',
                        markersize=markersize,color=cmap(ci),alpha=0.1)
                plt.plot(idx,raster[i,idx]*(count+1),
                        marker='s',linestyle='None',
                        markersize=markersize,color=cmap(ci),alpha=1)
                actividad[count]=np.sum(raster[i,:])*100/F
                count+=1      
        ax.set_xlim(0,F-1)
        ax.set_ylim(1,N)
        plt.xticks([])
        plt.ylabel("Etiqueta de Neurona")

        ax=plt.axes(((0.05,0.12,0.75,0.2)))
        coactividad=np.sum(raster,axis=0)
        fpm=fps*60
        tiempo=np.arange(0,F)/fpm

        coactividad_clusters=np.zeros((max(cluster_index),len(tiempo)))
        for ci in range(max(cluster_index)):
            coactividad_clusters[ci,:]=np.sum(raster[np.where(cluster_index==ci)[0],:],axis=0)

        if smoothing==False:
            for ci in range(max(cluster_index)):
                plt.plot(tiempo,coactividad_clusters[ci,:],linewidth=0.5,color=cmap(ci))
            plt.plot(tiempo,coactividad,linewidth=0.5,color='black')
        else:
            for ci in range(max(cluster_index)):
                plt.plot(tiempo,smooth(coactividad_clusters[ci,:],3),linewidth=0.5,color=cmap(ci))
            coactividad_smooth=smooth(coactividad,3)
            plt.plot(tiempo,coactividad_smooth,linewidth=0.5,color='black')
        ax.set_xlim(np.min(tiempo),np.max(tiempo))
        ax.set_ylim(0,np.max(coactividad)+1)
        plt.xlabel("Tiempo (min)")
        plt.ylabel("Coactividad")

        ax=plt.axes((0.85,0.35,0.1,0.6))
        plt.plot(actividad,np.arange(1,N+1),color='black',linewidth=1)
        ax.set_xlim(0,max(actividad)+1)
        ax.set_ylim(1,N)
        plt.xlabel("Cuadros activos")
        plt.yticks([])
    else:
        print("El tamaño del vector de picos significativos no coincide con el número de grupos")
    return orden_plot

In [ ]:
def extraer_componente_gigante(grafo):
    comps=buscar_componentes(grafo)
    comp_g=comps[0]
    mayor_valor=len(comps[0])
    for comp in comps:
        if len(comp)>mayor_valor:
            comp_g=comp
            mayor_valor=len(comp)
    nodos_cg=np.sort(np.asarray(comp_g))
    return grafo[nodos_cg[:,np.newaxis],nodos_cg]

def buscar_componentes(grafo):
    N=len(grafo)
    visitados=[]
    componentes=[]
    for i in range(N):
        visitados.append(False)
    for i in range(N):
        if visitados[i]==False:
            componente=[]
            componentes.append(buscar_en_vecinos(componente,i,visitados,grafo))
    return componentes

def buscar_en_vecinos(componente,i,visitados,grafo):
    visitados[i]=True
    componente.append(i)
    vecinos=np.where(grafo[i,:]>0)[0]
    for j in vecinos:
        if visitados[j]==False:
            componente=buscar_en_vecinos(componente,j,visitados,grafo)
    return componente

In [ ]:
def delete_multiple_element(list_object, indices):
  indices = sorted(indices, reverse=True)
  for idx in indices:
        if idx < len(list_object):
            list_object.pop(idx)

In [ ]:
def get_grafo_coactividad(raster,n_sim,p):
  N,F=raster.shape
  grafo_ponderado=np.zeros((N,N))
  grafo_binario=np.zeros((N,N))

  mat_conteo=np.zeros((N,N))

  mat_coact_real=np.zeros((N,N))
  mat_coact_sub=np.zeros((N,N,n_sim))
    
  for i in range(F):
    neuronas_coactivas=np.where(raster[:,i]==1)[0]
    for j in neuronas_coactivas:
      for k in neuronas_coactivas:
        mat_coact_real[j,k]=mat_coact_real[j,k]+1
  for i in range(N):
    mat_coact_real[i,i]=0

  for n in range(n_sim):
    raster_sub=raster_subrogado(raster,1)
    mat_coact_sub=np.zeros((N,N))
    for i in range(F):
      neuronas_coactivas=np.where(raster_sub[:,i]==1)[0]
      for j in neuronas_coactivas:
        for k in neuronas_coactivas:
          mat_coact_sub[j,k,n]=mat_coact_sub[j,k,n]+1
    for i in range(N):
      for j in range(N):
        if mat_coact_real[i,j]>mat_coact_sub[i,j,n]:
          mat_conteo[i,j]=mat_conteo[i,j]+1
  umbral=1-p
  for i in range(N):
    for j in range(N):
      if mat_conteo[i,j]>(n_sim*umbral):
        grafo_ponderado[i,j]=mat_coact_real[i,j]
        grafo_binario[i,j]=1
  
  return grafo_binario,grafo_ponderado

In [ ]:
def get_grafo_correlaciones(raster,n_sim,p):
  N,F=raster.shape
  grafo_ponderado=np.zeros((N,N))
  grafo_binario=np.zeros((N,N))

  mat_conteo=np.zeros((N,N))

  mat_corr_real=np.corrcoef(raster)
  mat_corr_real[np.where(mat_corr_real<0)]=0
    
  mat_corr_sub_total=np.zeros((N,N,n_sim))

  for n in range(n_sim):
    raster_sub=raster_subrogado(raster,1)
    mat_corr_sub=np.corrcoef(raster_sub)
    #mat_corr_sub[np.where(mat_corr_sub<0)]=0
    np.fill_diagonal(mat_corr_sub, 0)
    mat_corr_sub_total[:,:,n]=mat_corr_sub
    a,b=np.where(mat_corr_real>mat_corr_sub)
    mat_conteo[a,b]=mat_conteo[a,b]+1

  np.fill_diagonal(mat_corr_real, 0)
  umbral=1-p
  grafo_binario[np.where(mat_conteo>(n_sim*umbral))]=1
  grafo_ponderado=np.corrcoef(raster)
  #grafo_ponderado[np.where(mat_conteo<=(n_sim*umbral))]=0
  
  return grafo_binario,grafo_ponderado,mat_corr_sub_total

In [ ]:
def cargar_experimento(nombre_archivo):
  datos=pd.read_csv(nombre_archivo,sep=',',skiprows=4,header=None)
  return datos.values.T

In [ ]:
def exportar_json(datos,nombre_archivo):
  nombre_archivo=os.path.splitext(nombre_archivo)[0]+'.json'
  with open(nombre_archivo, "w") as outfile: 
    json.dump(datos, outfile)

In [ ]:
def test_coactividad_significativa(raster_real,n_iter,p,graficar=True):
    coactividad_real=np.sum(raster_real,axis=0)
    frecuencia_coactividad=collections.Counter(coactividad_real)
    lista_coactividad_subrogada=[]
    lista_hist_subrogada=[]
    for i in range(n_iter):
        raster_sub=raster_subrogado(raster_real,1)
        coactividad_subrogada=np.sum(raster_sub,axis=0)
        frecuencia_coactividad_subrogada=collections.Counter(coactividad_subrogada.astype(int))
        lista_hist_subrogada.append(frecuencia_coactividad_subrogada)
    probabilidades_coactividad=[]

    # cuentas=np.zeros((len(frecuencia_coactividad),))
    # for sim in lista_hist_subrogada:
    #     for i in range(len(frecuencia_coactividad)):
    #         if frecuencia_coactividad[i]>=sim[i]:
    #             cuentas[i]=cuentas[i]+1

    tam_picos=np.sort(list(frecuencia_coactividad.keys()))
    
#     plt.figure()
#     plt.plot(tam_picos,1-cuentas/n_iter,marker='o')
#     plt.plot([min(tam_picos),max(tam_picos)],[p,p],color='red',linestyle=':')
#     plt.ylabel('p')
#     plt.xlabel('Neuronas activas en un frame')
#     plt.title('Montecarlo 1')
#     plt.show()

    cuentas2=np.zeros((len(frecuencia_coactividad),))
    for sim in lista_hist_subrogada:
        for i in range(len(frecuencia_coactividad)):
            suma_real=0
            suma_subr=0
            for j in range(i,len(frecuencia_coactividad)):
                suma_real+=frecuencia_coactividad[j]
            for j in range(i,len(sim)):
                suma_subr+=sim[j]
            if suma_subr>suma_real:
                cuentas2[i]=cuentas2[i]+1
    
    if graficar:
        plt.figure()
        plt.plot(tam_picos,cuentas2/n_iter,marker='o')
        plt.plot([min(tam_picos),max(tam_picos)],[p,p],color='red',linestyle=':')
        plt.ylabel('p')
        plt.xlabel('Neuronas activas en un frame')
        plt.title('Método de Montecarlo')
        plt.show()
    
    return [tam_picos,cuentas2/n_iter,lista_hist_subrogada]

In [ ]:
def raster_subrogado(raster_real,selector):
  N,F=raster_real.shape
  raster_artificial=np.zeros((N,F))
  if selector == 1: #Mantiene número de neuronas y tiempo
    for i in range(N):
      raster_artificial[i,:]=raster_real[i,np.random.permutation(F)]
  elif selector == 2: #Mantiene ISI y tiempo
    for i in range(N):
      isi=np.diff(np.where(np.concatenate(([1],raster_real[i,:],[1]))==1)).flatten()
      t_inicio=isi[0]
      t_final=isi[-1]
      isi=np.delete(isi,[0])
      isi=np.delete(isi,len(isi)-1)
      isi_permutado=isi[np.random.permutation(len(isi))]
      t_prim_esp=np.random.randint(t_inicio+t_final)
      t_activo=np.concatenate(([0],np.cumsum(isi_permutado)))
      t_esp=t_activo+t_prim_esp-1
      raster_artificial[i,t_esp]=1
  return raster_artificial

In [ ]:
def get_actividad_acumulada(raster,fps,Normalizar=False):
  _,F=raster.shape
  fpm=fps*60
  tiempo=np.arange(0,F)/fpm
  if Normalizar:
    texto="Actividad Acumulada Normalizada"
    actividad_acumulada=np.cumsum(np.sum(raster,axis=0))/np.sum(np.sum(raster,axis=0))
  else:
    texto="Actividad Acumulada"
    actividad_acumulada=np.cumsum(np.sum(raster,axis=0))
  m,b = np.polyfit(tiempo,actividad_acumulada, 1)
  y=m*tiempo+b
  plt.plot(tiempo,actividad_acumulada)
  plt.plot(tiempo,y,'--')
  plt.xlim(np.min(tiempo),np.max(tiempo))
  plt.ylim(0,max(actividad_acumulada))
  plt.xlabel("Tiempo (min)")
  plt.ylabel(texto)
  return m

In [ ]:
def plot_raster(raster,fps,smoothing=False,umbral=None,markersize=5):
    N,F=raster.shape
    plt.figure(figsize=(12,6))

    ax=plt.axes((0.05,0.35,0.75,0.6)) 
    for i in range(N):
        indices=np.where(raster[i,:]==1)[0]
        plt.plot(indices,raster[i,indices]*(i+1),
                marker='|',linestyle='None',
                markersize=markersize,color='black')
    ax.set_xlim(0,F-1)
    ax.set_ylim(1,N)
    plt.xticks([])
    plt.ylabel("Etiqueta de Neurona")

    ax=plt.axes(((0.05,0.12,0.75,0.2)))
    coactividad=np.sum(raster,axis=0)
    coactividad_smooth=smooth(coactividad,3)
    fpm=fps*60
    tiempo=np.arange(0,F)/fpm
    if smoothing==False:
        plt.plot(tiempo,coactividad,linewidth=0.5,color='black')
    else:
        plt.plot(tiempo,coactividad_smooth,linewidth=0.5,color='black')
    if umbral!=None:
        plt.plot([np.min(tiempo),np.max(tiempo)],[umbral-0.2,umbral-0.2],linestyle='--',color='black',linewidth=1)
    ax.set_xlim(np.min(tiempo),np.max(tiempo))
    ax.set_ylim(0,np.max(coactividad)+1)
    plt.xlabel("Tiempo (min)")
    plt.ylabel("Coactividad")

    ax=plt.axes((0.85,0.35,0.1,0.6))
    actividad=np.sum(raster,axis=1)
    plt.plot(actividad,np.arange(1,N+1),color='black',linewidth=1)
    ax.set_xlim(0,max(actividad)+1)
    ax.set_ylim(1,N)
    plt.xlabel("Cuadros activos")
    plt.yticks([])

In [ ]:
def plot_raster_coactividad_significativa(raster,fps,umbral_coactividad):
  N,F=raster.shape
  plt.figure(figsize=(12,6))

  f_coac_sig=np.where(np.sum(raster,axis=0)>=umbral_coactividad)[0]
  raster_atras=raster.copy()
  raster_atras[:,f_coac_sig]=0
  f_no_coac_sig=np.where(np.sum(raster,axis=0)<umbral_coactividad)[0]
  raster_adelante=raster.copy()
  raster_adelante[:,f_no_coac_sig]=0

  ax=plt.axes((0.05,0.35,0.75,0.6)) 
  for i in range(N):
    indices=np.where(raster_atras[i,:]==1)[0]
    plt.plot(indices,raster_atras[i,indices]*(i+1),
            marker='|',linestyle='None',
            markersize=1,color='black')
    indices=np.where(raster_adelante[i,:]==1)[0]
    plt.plot(indices,raster_adelante[i,indices]*(i+1),
            marker='o',linestyle='None',
            markersize=2,color='red')
  ax.set_xlim(0,F-1)
  ax.set_ylim(1,N)
  plt.xticks([])
  plt.ylabel("Etiqueta de Neurona")

  ax=plt.axes(((0.05,0.12,0.75,0.2)))
  y=np.sum(raster,axis=0)
  fpm=fps*60
  x=np.arange(0,F)/fpm
  xvals = np.linspace(min(x), max(x), len(x)*10)
  yinterp = np.interp(xvals, x, y)
  y_masked = np.ma.masked_where(yinterp < (umbral_coactividad-0.5), yinterp)
  plt.plot(xvals,yinterp,linewidth=0.5,color='black')
  plt.plot(xvals,y_masked,linewidth=2,color='red')
  ax.set_xlim(np.min(tiempo),np.max(tiempo))
  ax.set_ylim(0,np.max(y)+1)
  plt.xlabel("Tiempo (min)")
  plt.ylabel("Coactividad")

  ax=plt.axes((0.85,0.35,0.1,0.6))
  actividad=np.sum(raster,axis=1)
  base=plt.gca().transData
  rot=tfs.Affine2D().rotate_deg(270)
  plt.plot(np.flip(actividad),color='black',
          linewidth=1,transform=rot+base)
  plt.xlabel("Cuadros activos")
  plt.yticks([])
  ax.set_xlim(0,np.max(actividad)+1)
  ax.set_ylim(-N+1,0)

In [ ]:
def teachers_round(x):
    '''
    Do rounding such that .5 always rounds to 1, and not bankers rounding.
    This is for compatibility with matlab functions, and ease of testing.
    '''
    if ((x > 0) and (x % 1 >= 0.5)) or ((x < 0) and (x % 1 > 0.5)):
        return int(np.ceil(x))
    else:
        return int(np.floor(x))

In [ ]:
def threshold_proportional(W, p, copy=True):
    '''
    This function "thresholds" the connectivity matrix by preserving a
    proportion p (0<p<1) of the strongest weights. All other weights, and
    all weights on the main diagonal (self-self connections) are set to 0.
    If copy is not set, this function will *modify W in place.*
    Parameters
    ----------
    W : np.ndarray
        weighted connectivity matrix
    p : float
        proportional weight threshold (0<p<1)
    copy : bool
        if True, returns a copy of the matrix. Otherwise, modifies the matrix
        in place. Default value=True.
    Returns
    -------
    W : np.ndarray
        thresholded connectivity matrix
    Notes
    -----
    The proportion of elements set to 0 is a fraction of all elements
    in the matrix, whether or not they are already 0. That is, this function
    has the following behavior:
    >> x = np.random.random_sample((10,10))
    >> x_25 = threshold_proportional(x, .25)
    >> np.size(np.where(x_25)) #note this double counts each nonzero element
    46
    >> x_125 = threshold_proportional(x, .125)
    >> np.size(np.where(x_125))
    22
    >> x_test = threshold_proportional(x_25, .5)
    >> np.size(np.where(x_test))
    46
    That is, the 50% thresholding of x_25 does nothing because >=50% of the
    elements in x_25 are aleady <=0. This behavior is the same as in BCT. Be
    careful with matrices that are both signed and sparse.
    '''
    if copy:
        W = W.copy()
    n = len(W) # number of nodes
    np.fill_diagonal(W, 0) # clear diagonal

    if np.allclose(W, W.T): # if symmetric matrix
        W[np.tril_indices(n)] = 0 # ensure symmetry is preserved
        ud = 2 # halve number of removed links
    else:
        ud = 1

    ind = np.where(W) # find all links

    I = np.argsort(W[ind])[::-1] # sort indices by magnitude

    en = int(teachers_round((n * n - n) * p / ud)) # number of links to be preserved

    W[(ind[0][I][en:], ind[1][I][en:])] = 0  # apply threshold
    #W[np.ix_(ind[0][I][en:], ind[1][I][en:])]=0

    if ud == 2: # if symmetric matrix
        W[:, :] = W + W.T # reconstruct symmetry

    return W

In [ ]:
def community_louvain(W, gamma=1, ci=None, B='modularity', seed=None):
    '''
    The optimal community structure is a subdivision of the network into
    nonoverlapping groups of nodes which maximizes the number of within-group
    edges and minimizes the number of between-group edges.
    This function is a fast an accurate multi-iterative generalization of the
    louvain community detection algorithm. This function subsumes and improves
    upon modularity_[louvain,finetune]_[und,dir]() and additionally allows to
    optimize other objective functions (includes built-in Potts Model i
    Hamiltonian, allows for custom objective-function matrices).
    Parameters
    ----------
    W : NxN np.array
        directed/undirected weighted/binary adjacency matrix
    gamma : float
        resolution parameter. default value=1. Values 0 <= gamma < 1 detect
        larger modules while gamma > 1 detects smaller modules.
        ignored if an objective function matrix is specified.
    ci : Nx1 np.arraylike
        initial community affiliation vector. default value=None
    B : str | NxN np.arraylike
        string describing objective function type, or provides a custom
        NxN objective-function matrix. builtin values 
            'modularity' uses Q-metric as objective function
            'potts' uses Potts model Hamiltonian.
            'negative_sym' symmetric treatment of negative weights
            'negative_asym' asymmetric treatment of negative weights
    seed : hashable, optional
        If None (default), use the np.random's global random state to generate random numbers.
        Otherwise, use a new np.random.RandomState instance seeded with the given value.
    Returns
    -------
    ci : Nx1 np.array
        final community structure
    q : float
        optimized q-statistic (modularity only)
    '''
    rng = get_rng(seed)
    n = len(W)
    s = np.sum(W)

    #if np.min(W) < -1e-10:
    #    raise BCTParamError('adjmat must not contain negative weights')

    if ci is None:
        ci = np.arange(n) + 1
    else:
        if len(ci) != n:
            raise BCTParamError('initial ci vector size must equal N')
        _, ci = np.unique(ci, return_inverse=True)
        ci += 1
    Mb = ci.copy()
    renormalize = False
    if B in ('negative_sym', 'negative_asym'):
        renormalize = True
        W0 = W * (W > 0)
        s0 = np.sum(W0)
        B0 = W0 - gamma * np.outer(np.sum(W0, axis=1), np.sum(W0, axis=0)) / s0

        W1 = -W * (W < 0)
        s1 = np.sum(W1)
        if s1:
            B1 = W1 - gamma * np.outer(np.sum(W1, axis=1), np.sum(W1, axis=0)) / s1
        else:
            B1 = 0

    elif np.min(W) < -1e-10:
        raise BCTParamError("Input connection matrix contains negative "
            'weights but objective function dealing with negative weights '
            'was not selected')

    if B == 'potts' and np.any(np.logical_not(np.logical_or(W == 0, W == 1))):
        raise BCTParamError('Potts hamiltonian requires binary input matrix')

    if B == 'modularity':
        B = W - gamma * np.outer(np.sum(W, axis=1), np.sum(W, axis=0)) / s
    elif B == 'potts':
        B = W - gamma * np.logical_not(W)
    elif B == 'negative_sym':
        B = (B0 / (s0 + s1)) - (B1 / (s0 + s1))
    elif B == 'negative_asym':
        B = (B0 / s0) - (B1 / (s0 + s1))
    else:
        try:
            B = np.array(B)
        except:
            raise BCTParamError('unknown objective function type')

        if B.shape != W.shape:
            raise BCTParamError('objective function matrix does not match '
                                'size of adjacency matrix')
        if not np.allclose(B, B.T):
            print ('Warning: objective function matrix not symmetric, '
                   'symmetrizing')
            B = (B + B.T) / 2
    
    Hnm = np.zeros((n, n))
    for m in range(1, n + 1):
        Hnm[:, m - 1] = np.sum(B[:, ci == m], axis=1)  # node to module degree
    H = np.sum(Hnm, axis=1)  # node degree
    Hm = np.sum(Hnm, axis=0)  # module degree

    q0 = -np.inf
    # compute modularity
    q = np.sum(B[np.tile(ci, (n, 1)) == np.tile(ci, (n, 1)).T]) / s

    first_iteration = True

    while q - q0 > 1e-10:
        it = 0
        flag = True
        while flag:
            it += 1
            if it > 1000:
                raise BCTParamError('Modularity infinite loop style G. '
                                    'Please contact the developer.')
            flag = False
            for u in rng.permutation(n):
                ma = Mb[u] - 1
                dQ = Hnm[u, :] - Hnm[u, ma] + B[u, u]  # algorithm condition
                dQ[ma] = 0

                max_dq = np.max(dQ)
                if max_dq > 1e-10:
                    flag = True
                    mb = np.argmax(dQ)

                    Hnm[:, mb] += B[:, u]
                    Hnm[:, ma] -= B[:, u]  # change node-to-module strengths

                    Hm[mb] += H[u]
                    Hm[ma] -= H[u]  # change module strengths

                    Mb[u] = mb + 1

        _, Mb = np.unique(Mb, return_inverse=True)
        Mb += 1

        M0 = ci.copy()
        if first_iteration:
            ci = Mb.copy()
            first_iteration = False
        else:
            for u in range(1, n + 1):
                ci[M0 == u] = Mb[u - 1]  # assign new modules

        n = np.max(Mb)
        b1 = np.zeros((n, n))
        for i in range(1, n + 1):
            for j in range(i, n + 1):
                # pool weights of nodes in same module
                bm = np.sum(B[np.ix_(Mb == i, Mb == j)])
                b1[i - 1, j - 1] = bm
                b1[j - 1, i - 1] = bm
        B = b1.copy()

        Mb = np.arange(1, n + 1)
        Hnm = B.copy()
        H = np.sum(B, axis=0)
        Hm = H.copy()

        q0 = q

        q = np.trace(B)  # compute modularity
    
    # Workaround to normalize
    if not renormalize:
        return ci, q/s
    else:
        return ci, q